In [106]:
import geemap
import ee
import os
ee.Initialize(project=	'shadesync-477715')

In [107]:
Map = geemap.Map()
Map = geemap.Map(
    center=[13.743240997587234, 100.56562878077352], 
    zoom=19, 
     attribution=False,
    toolbar_ctrl=False, 
    layer_ctrl=False,
    draw_ctrl=False,
    zoom_ctrl=False,
    scale_ctrl=False,
    fullscreen_ctrl=False,
    search_ctrl = False,
    measure_ctrl = False,
)
Map.add_basemap("Esri.WorldImagery") 

In [66]:
Map

Map(center=[13.743240997587234, 100.56562878077352], controls=(WidgetControl(options=['position', 'transparent…

In [122]:
import requests
import os

# Center point
lat, lon = 13.74, 100.56

zoom_in = 0.0030 #0.0030
bbox = f"{lon-zoom_in},{lat-zoom_in},{lon+zoom_in},{lat+zoom_in}"
size = 2048  # pixels for sharper image

# ESRI World Imagery static export
url = f"https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?bbox={bbox}&bboxSR=4326&size={size},{size}&format=png&f=image"

r = requests.get(url)

download_dir = os.path.join(os.path.expanduser("~"), "Downloads")
os.makedirs(download_dir, exist_ok=True)
output_path = os.path.join(download_dir, "bangkok_esri_zoomed.png")

with open(output_path, "wb") as f:
    f.write(r.content)

print("Saved:", output_path)


Saved: /home/winkawin2552/Downloads/bangkok_esri_zoomed.png


In [119]:
import requests
import os

# --- Configuration ---
# Center point for the first image
lat, lon = 13.74, 100.56

zoom_in = 0.0030
size = 2048 # pixels for sharper image

# Calculate the distance to shift the center point for the next block
# A single block's width is 2 * zoom_in
shift_amount = 2 * zoom_in 

download_dir = os.path.join(os.path.expanduser("~"), "Downloads")
os.makedirs(download_dir, exist_ok=True)
# ---------------------

def save_map_image(center_lat, center_lon, filename_suffix):
    """Fetches and saves the satellite image for a given center point."""
    
    # 1. Create the Bounding Box (BBOX) for the new center
    bbox = f"{center_lon - zoom_in},{center_lat - zoom_in},{center_lon + zoom_in},{center_lat + zoom_in}"
    
    # 2. Construct the URL
    url = f"https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?bbox={bbox}&bboxSR=4326&size={size},{size}&format=png&f=image"
    
    # 3. Make the Request
    r = requests.get(url)

    # 4. Save the File
    output_path = os.path.join(download_dir, f"bangkok_esri_{filename_suffix}.png")
    with open(output_path, "wb") as f:
        f.write(r.content)

    print(f"Saved: {output_path}")

# --- Generate Image 1 (Original Center) ---
save_map_image(lat, lon, "original")

# --- Generate Image 2 (Shifted Right) ---
# Move the center longitude to the right by one block's width.
lon_shifted = lon + shift_amount 
save_map_image(lat, lon_shifted, "shifted_right")

Saved: /home/winkawin2552/Downloads/bangkok_esri_original.png
Saved: /home/winkawin2552/Downloads/bangkok_esri_shifted_right.png


# START CAPTURE IMAGE

In [121]:
import requests
import os
import time

# --- Configuration ---

# 1. Center point for the top-left image (Row 0, Column 0)
START_LAT = 13.74
START_LON = 100.56

# 2. Zoom level and image size
ZOOM_IN = 0.0030
IMAGE_SIZE = 2048 # pixels for sharper image

# 3. Grid dimensions (10 columns wide, 5 rows high = 50 images total)
NUM_COLUMNS = 10
NUM_ROWS = 5

# 4. Target Directory (ABSOLUTE PATH provided by user)
DOWNLOAD_DIR = "/home/winkawin2552/Code/ShadeSync/get_data_for_train/img"

# --- Timeout and Retry Settings ---
MAX_RETRIES = 3         # Max number of attempts per image
TIMEOUT_SECONDS = 30    # Increased timeout from 10s to 30s
RETRY_DELAY = 0.5       # Base delay for retries (uses exponential backoff)

# Calculated shift amount (The geographic width/height of one block)
SHIFT_AMOUNT = 2 * ZOOM_IN 

# Create the directory if it doesn't exist
try:
    os.makedirs(DOWNLOAD_DIR, exist_ok=True)
except Exception as e:
    print(f"Error creating directory {DOWNLOAD_DIR}: {e}")
    exit()
# ---------------------

def save_map_image(center_lat, center_lon, filename):
    """Fetches and saves the satellite image for a given center point, with retries."""
    
    # 1. Create the Bounding Box (BBOX)
    bbox = f"{center_lon - ZOOM_IN},{center_lat - ZOOM_IN},{center_lon + ZOOM_IN},{center_lat + ZOOM_IN}"
    
    # 2. Construct the URL
    url = (
        f"https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?"
        f"bbox={bbox}&bboxSR=4326&size={IMAGE_SIZE},{IMAGE_SIZE}&format=png&f=image"
    )
    
    # 3. Implement Retry Loop
    for attempt in range(MAX_RETRIES):
        try:
            r = requests.get(url, timeout=TIMEOUT_SECONDS)
            r.raise_for_status() # Raise HTTPError for bad status codes (e.g., 404, 500)
            
            # If successful, break the retry loop and proceed to save
            output_path = os.path.join(DOWNLOAD_DIR, filename)
            with open(output_path, "wb") as f:
                f.write(r.content)

            print(f"Successfully saved: {output_path}")
            return # Exit the function on success
            
        except requests.exceptions.RequestException as e:
            # Handle timeout or other request errors
            if attempt < MAX_RETRIES - 1:
                # Exponential backoff: Wait longer on subsequent failures (0.5s, 1s, 2s)
                wait_time = RETRY_DELAY * (2 ** attempt) 
                print(f"Attempt {attempt + 1}/{MAX_RETRIES} for {filename} failed: {e}. Retrying in {wait_time:.1f}s...")
                time.sleep(wait_time)
            else:
                print(f"Failed to download {filename} after {MAX_RETRIES} attempts. Giving up.")
                return # Exit the function on final failure


print(f"Starting image download grid: {NUM_COLUMNS} columns x {NUM_ROWS} rows ({NUM_COLUMNS * NUM_ROWS} images total)...")
print("-" * 50)

# Nested loops to iterate through the grid
for r in range(NUM_ROWS):
    for c in range(NUM_COLUMNS):
        
        # Calculate the shift for this position
        lon_shift = c * SHIFT_AMOUNT
        lat_shift = r * SHIFT_AMOUNT
        
        # Calculate the center point for the current tile
        current_lon = START_LON + lon_shift
        current_lat = START_LAT - lat_shift 
        
        filename = f"tile_R{r:02d}_C{c:02d}.png"
        
        # Save the image
        save_map_image(current_lat, current_lon, filename)
        
        # Add a deliberate delay between requests to be less aggressive
        time.sleep(0.5) 

print("-" * 50)
print("Download complete.")

Starting image download grid: 10 columns x 5 rows (50 images total)...
--------------------------------------------------
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C00.png
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C01.png
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C02.png
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C03.png
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C04.png
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C05.png
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C06.png
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C07.png
Successfully saved: /home/winkawin2552/Code/ShadeSync/get_data_for_train/img/tile_R00_C08.png
Successfully saved: /home/winkaw